In [32]:
# data wrangling
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [24]:
all_data = pd.read_csv("../geotracker/data/all_data.csv").iloc[:, 1:]
# converting lat/lon & coords into 5 decimal figures
all_data["latitude"] = all_data.latitude.round(5)
all_data["longitude"] = all_data.longitude.round(5)
all_data["coordinates"] = all_data.latitude.astype(
    str) + "," + all_data.longitude.astype(str)


In [25]:
all_data.head()

,restaurant_name,avg_review_score,reviews,minimum_order_value,delivery_fee,pricyness,avg_delivery_time,type_of_cuisine,address,street,zip_code,city_name,coordinates,latitude,longitude,database,type,opening_hours
0,BunUp Mitte,7.8,NaN,10.0,NaN,1.0,35.0,vegetarian or vegan,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.52383,13.39857",52.52383,13.39857,wolt,NaN,NaN
1,Machiavelli,8.4,NaN,10.0,1.9,2.0,NaN,european,"Albrechtstr.13 Berlin, 10117.0, Berlin",Albrechtstr.13 Berlin,10117.0,Berlin,"52.5213,13.38518",52.52130,13.38518,wolt,NaN,NaN
2,Pizza Peppino,NaN,NaN,10.0,NaN,1.0,35.0,european,"Tucholskystraße 13, 10117.0, Berlin",Tucholskystraße 13,10117.0,Berlin,"52.52339,13.39176",52.52339,13.39176,wolt,NaN,NaN
3,Fresh's,7.8,NaN,10.0,NaN,1.0,35.0,poke,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.52383,13.39857",52.52383,13.39857,wolt,NaN,NaN
4,Bombay,8.4,NaN,10.0,NaN,2.0,35.0,middle eastern,"Friedrichstraße 106c, 10117.0, Berlin",Friedrichstraße 106c,10117.0,Berlin,"52.52327,13.38846",52.52327,13.38846,wolt,NaN,NaN


In [55]:
# dividing all data into 3 dataframes; each for one column

wolt = all_data[all_data.database == "wolt"]
lieferando = all_data[all_data.database == "lieferando"]
maps = all_data[all_data.database == "here_maps"]

prueba_maps = maps.loc[:500]
prueba_wolt = wolt.loc[:500]
prueba_liefe = lieferando.loc[:500]

In [49]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['match'] = m

    m2 = df_1['match'].apply(
        lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['match'] = m2

    return df_1

In [57]:




fuzzy_merge(maps,
            prueba_liefe,
            "restaurant_name",
            "restaurant_name",
            threshold=90,
            limit=1)


/var/folders/g9/cc5vy2_x75l91yh_4j05j5_80000gn/T/ipykernel_2341/1912964862.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match'] = m
/var/folders/g9/cc5vy2_x75l91yh_4j05j5_80000gn/T/ipykernel_2341/1912964862.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['match'] = m2


,restaurant_name,avg_review_score,reviews,minimum_order_value,delivery_fee,pricyness,avg_delivery_time,type_of_cuisine,address,street,zip_code,city_name,coordinates,latitude,longitude,database,type,opening_hours,match
4683,Ihr Koch on Tour,NaN,NaN,NaN,NaN,NaN,NaN,european,Seelenbinderstraße 112 Köpenick 12555 Berlin,Seelenbinderstraße 112,12555,Köpenick,"52.45481,13.59031",52.45481,13.59031,here_maps,Restaurant,Wed-Sat: 17:00 - 22:30 Sun: 14:00 - 19:00,
4684,Pizzawerkköpenick,NaN,NaN,NaN,NaN,NaN,NaN,italian,Seelenbinderstraße 112 Köpenick 12555 Berlin,Seelenbinderstraße 112,12555,Köpenick,"52.45481,13.59031",52.45481,13.59031,here_maps,Restaurant,NaN,
4685,Pizzawerk,NaN,NaN,NaN,NaN,NaN,NaN,italian,Seelenbinderstraße 112 Köpenick 12555 Berlin,Seelenbinderstraße 112,12555,Köpenick,"52.45481,13.59031",52.45481,13.59031,here_maps,Restaurant,Tue-Sun: 17:00 - 22:00,
4686,Veracruz,NaN,NaN,NaN,NaN,NaN,NaN,mexican,Fürstenwalder Damm 260 Friedrichshagen 12587 B...,Fürstenwalder Damm 260,12587,Friedrichshagen,"52.45268,13.59594",52.45268,13.59594,here_maps,Restaurant,"Mon-Thu, Sun: 12:00 - 23:00 Fri, Sat: 12:00 - ...",
4687,Marina Sol,NaN,NaN,NaN,NaN,NaN,NaN,european,Müggelseedamm 70 Friedrichshagen 12587 Berlin,Müggelseedamm 70,12587,Friedrichshagen,"52.44842,13.61015",52.44842,13.61015,here_maps,Restaurant,Mon-Sun: 12:00 - 22:00,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8025,Restaurant Maracas,NaN,NaN,NaN,NaN,NaN,NaN,steak,Neuköllner Straße 201 Rudow 12357 Berlin,Neuköllner Straße 201,12357,Rudow,"52.43003,13.47804",52.43003,13.47804,here_maps,Restaurant,Mon-Sun: 12:00 - 22:00,
8026,Verde's Pizza & Gelato,NaN,NaN,NaN,NaN,NaN,NaN,italian,JoachimGottschalkWeg 21 Gropiusstadt 12353 Berlin,JoachimGottschalkWeg 21,12353,Gropiusstadt,"52.42294,13.47549",52.42294,13.47549,here_maps,Restaurant,Mon-Sat: 09:00 - 22:00,
8027,Eiscafé Kunterbunt,NaN,NaN,NaN,NaN,NaN,NaN,breakfast/dessert,Waldsassener Straße 59A Marienfelde 12279 Berlin,Waldsassener Straße 59A,12279,Marienfelde,"52.40663,13.34947",52.40663,13.34947,here_maps,Food & Drink,Mon-Sun: 12:00 - 19:00,
8028,Osteria Ballerino,NaN,NaN,NaN,NaN,NaN,NaN,mediterranean,Schwanenwerderweg Nikolassee 14129 Berlin,Schwanenwerderweg,14129,Nikolassee,"52.44167,13.18912",52.44167,13.18912,here_maps,Restaurant,Mon-Sun: 10:00 - 22:00,


In [59]:
maps.match.unique()

array([''], dtype=object)